In [2]:
import os
import requests
import urllib.parse
import datetime
import pandas as pds

In [3]:
def datetime_converter(type, time):
    if type == 'date':
        timestamp = int(time) / 1000
        date = datetime.datetime.fromtimestamp(timestamp)
        return date
    elif type == 'timestamp':
        date = datetime.datetime.strptime(time, '%Y-%m-%d')
        timestamp = int(date.timestamp())
        return timestamp
    else:
        raise ValueError("Invalid type. Supported types are 'date' and 'timestamp'.")


In [5]:
def get_puuid(game_name, name_tag):
    with open('../api_key.txt', 'r') as file:
        api_key = file.read().replace('\n', '')
    url_puuid = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{name_tag}?api_key={api_key}'
    response_puuid = requests.get(url_puuid)
    return response_puuid.json()['puuid']

In [6]:
def get_matchids(puuid, amount_of_games=10000, starttime=0, endtime=0, type='ranked'):
    with open('../api_key.txt', 'r') as file:
        api_key = file.read().replace('\n', '')
    
    match_ids = []
    count = 0
    start = 0
    
    while count < amount_of_games:
        remaining_games = amount_of_games - count
        games_to_fetch = min(remaining_games, 100)
        
        if starttime == 0 and endtime == 0:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        elif starttime != 0 and endtime == 0:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={starttime}&type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        else:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={starttime}&endTime={endtime}&type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        
        response_matchids = requests.get(url_matchids)
        match_ids.extend(response_matchids.json())
        
        count += games_to_fetch
        start += games_to_fetch
    
    return match_ids


In [7]:
def get_match_data(match_id):
    try:
        with open('../api_key.txt', 'r') as file:
            api_key = file.read().replace('\n', '')
        url_match = f'https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
        response_match = requests.get(url_match)
        response_json = response_match.json()
        
        if 'status' in response_json and response_json['status']['status_code'] == 404:
            print('Data not found - match file not found')
            return None
        
        return response_json
    except Exception as e:
        print(f'Error occurred: {str(e)}')
        return {}


In [8]:
def filter_data(data, my_puuid, puuids_to_exclude):
    filtered_data = []
    for match in data:
        my_summoner = match['metadata']['participants'].index(my_puuid)
        team_id_my_summoner = match['info']['participants'][my_summoner]['teamId']
        for participant in match['info']['participants']:
            if (participant['teamPosition'] == 'BOTTOM' or participant['teamPosition'] == 'UTILITY') and participant['teamId'] == team_id_my_summoner and participant['puuid'] != puuids_to_exclude:
                filtered_data.append(participant)
    return filtered_data

In [9]:
my_puuid = get_puuid('Zp4t', 'EUW')
puuids_to_exclude = [get_puuid(puuid[0], puuid[1]) for puuid in [('Lurrvpöse', 'EUW'), ('Deadlystream538', 'EUW')]]

match_ids = get_matchids(my_puuid, amount_of_games=50, type='ranked')
filtered_data = []

for match in match_ids:
    match_data = get_match_data(match)
    if match_data is not None:
        filtered_data.extend(filter_data([match_data], my_puuid, puuids_to_exclude))

In [10]:
filtered_data = pd.DataFrame(filtered_data)
kda = filtered_data[['kills', 'deaths', 'assists']]
kda['deaths'].replace(0, 1, inplace=True)
kda_mean = kda.mean()
print((kda_mean['kills'] + kda_mean['assists']) / kda_mean['deaths'])

2.1085972850678734


C:\Users\first\AppData\Local\Temp\ipykernel_17940\3418175646.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  kda['deaths'].replace(0, 1, inplace=True)
C:\Users\first\AppData\Local\Temp\ipykernel_17940\3418175646.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kda['deaths'].replace(0, 1, inplace=True)


: 